<a href="https://colab.research.google.com/github/LoveYouTenderly/LoveYouTenderly/blob/main/ADA_pkl_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Original Source
# https://www.youtube.com/watch?v=L3JLzoe-dJU
# https://github.com/jeffheaton/present/blob/master/youtube/gan/colab_gan_train.ipynb
# continue connection
# https://m.blog.naver.com/dsz08082/222019211109

In [ ]:
#for irl install
#https://docs.nvidia.com/cuda/cuda-installation-guide-microsoft-windows/index.html#uninstall-cuda-software
# https://github.com/NVlabs/stylegan2-ada-pytorch/issues/155

In [ ]:
# function ClickConnect(){
# console.log("Working"); 
# document.querySelector("colab-toolbar-button").click() 
# }setInterval(ClickConnect, 1800000)

**Set Up New Environment**

You will likely need to train for >24 hours. Colab will disconnect you. You must be prepared to restart training when this eventually happens. Training is divided into ticks, every so many ticks (50 by default) your neural network is evaluated and a snapshot is saved. When CoLab shuts down, all training after the last snapshot is lost. It might seem desirable to snapshot after each tick; however, this snapshotting process itself takes nearly an hour. It is important to learn an optimal snapshot size for your resolution and training data.

We will mount GDRIVE so that your snapshots are saved there. You must also place your training images in GDRIVE.

In [ ]:
from tqdm import tqdm
from PIL import Image
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files

Mounted at /content/drive


## base download

In [ ]:
!nvidia-smi

Wed May  4 09:01:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    31W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install torch==1.8.1 torchvision==0.9.1

     |████████████████████████████████| 804.1 MB 2.7 kB/s 
     |████████████████████████████████| 17.4 MB 567 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.12.0+cu113
    Uninstalling torchvision-0.12.0+cu113:
      Successfully uninstalled torchvision-0.12.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.12.0 requires torch==1.11.0, but you have torch 1.8.1 which is incompatible.
torchaudio 0.11.0+cu113 requires torch==1.11.0, but you have torch 1.8.1 which is incompatible.


In [ ]:
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git
!pip install ninja

Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 128, done.
remote: Total 128 (delta 0), reused 0 (delta 0), pack-reused 128
Receiving objects: 100% (128/128), 1.12 MiB | 7.31 MiB/s, done.
Resolving deltas: 100% (57/57), done.
     |████████████████████████████████| 108 kB 5.6 MB/s 


In [ ]:
TARGET_PATH = "/content/drive/MyDrive/Image_making/Abstract/flower"

In [ ]:
import os
import glob
from PIL import Image
len(glob.glob(os.path.join(TARGET_PATH,"*")))

0

## **Find Your Files & Convert Your Images**

In [ ]:
!ls {TARGET_PATH}

FL-0002fe490d92aa39b2ad102b3d6a649240f4123cc62dd2f558cdab308ffecf5b.jpg
FL-000858da43dba85a3c78186da1cd85ef977eeb724af4a0c21933c974a855deb6.jpg
FL-0022697c1a2d93e6b09a0a1ae465ab2e27f83fd1e9c7c184a8157d64e7122257.jpg
FL-0023525553eff5ca563dec3728ab632304a19aa0ca462ce3369d01cd66194e02.jpg
FL-00328e37eb6c96116b875f3ef620b49ba486e34eb8d95b64847ad37e68725d5b.jpg
FL-004a8ddf5355c6756eed874bc1e7165cdcefa46155666d8a017bd8221ccd6532.jpg
FL-005260382b3dc8445bec3913be8638f488565e24feca860a9f465593853a552b.jpg
FL-005909ef2c4cc16472d0c8b1153f74b26b513ac229157e090835a00a2c2bec35.jpg
FL-0070246ff7de3dd0994a6603db2c69db94439968914837af18f12fc0d51b50e3.jpg
FL-0070f301725d2dcf085a56af5194e0b8719f8b9e29d81b8ea8c02ca4a3b95881.jpg
FL-009b8f361e00968aa21ec92f46f850fe10710dd55ac531718543606247ccd028.jpg
FL-00ca02200254ab09a43a33fd793e96a5d89a6e872c2140fd98dc29dcc98afad0.jpg
FL-00f9ead47c105376830f1cd742eaad0aa23ee501cd834270ca7662f160e6260e.jpg
FL-013ea88ece54dc1bceadd1f9905b8170eda93388c65bc8b740607ea962fcb

In [ ]:
!python /content/stylegan2-ada-pytorch/dataset_tool.py --source {TARGET_PATH} --dest /content/drive/MyDrive/Image_making/Abstract/Dataflower

100% 3838/3838 [10:47<00:00,  5.93it/s]


In [ ]:
# Never used so far
#!rm -R /content/drive/MyDrive/data/gan/dataset/circuit/*

**Clean Up your Images**

It is important that all images have the same dimensions and color depth. This code can identify images that have issues.

In [ ]:
from os import listdir
from os.path import isfile, join
import os
from PIL import Image
from tqdm.notebook import tqdm

IMAGE_PATH = TARGET_PATH
files = [f for f in listdir(IMAGE_PATH) if isfile(join(IMAGE_PATH, f))]

base_size = None
for file in tqdm(files):
  file2 = os.path.join(IMAGE_PATH,file)
  img = Image.open(file2)
  sz = img.size
  if base_size and sz!=base_size:
    print(f"Inconsistant size: {file2}")
  elif img.mode!='RGB':
    print(f"Inconsistant color format: {file2}")
  else:
    base_size = sz

  0%|          | 0/917 [00:00<?, ?it/s]

## **Perform Initial Training**

In [ ]:
import os

# Modify these to suit your needs
EXPERIMENTS = "/content/drive/MyDrive/Image_making/Abstract/Exoflower"
DATA = TARGET_PATH
SNAP = 1

# Build the command and run it
cmd = f"/usr/bin/python3 /content/stylegan2-ada-pytorch/train.py --snap {SNAP} --outdir {EXPERIMENTS} --data {DATA}"
!{cmd}


Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 1,
  "network_snapshot_ticks": 1,
  "metrics": [
    "fid50k_full"
  ],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "/content/drive/MyDrive/Image_making/Abstract/flower",
    "use_labels": false,
    "max_size": 3838,
    "xflip": false,
    "resolution": 1024
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "synthesis_kwargs": {
      "channel_base": 32768,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 32

In [ ]:
!/usr/bin/python3 /content/stylegan2-ada-pytorch/train.py --snap 10 --resume /content/drive/MyDrive/data/gan/experiments/00007-circuit-auto1/network-snapshot-000500.pkl --outdir /content/drive/MyDrive/data/gan/experiments --data /content/drive/MyDrive/data/gan/dataset/circuit

## **Resume Training**

In [ ]:
import os

# Modify these to suit your needs
EXPERIMENTS = "/content/drive/MyDrive/Image_making/texture/Marble_data_Exp"
DATA = TARGET_PATH
NETWORK = "network-snapshot-000004.pkl"
RESUME = os.path.join(EXPERIMENTS, "00048-Marble_data_convert_1-auto1-resumecustom", NETWORK)
SNAP = 1

# Build the command and run it
cmd = f"/usr/bin/python3 /content/stylegan2-ada-pytorch/train.py --snap {SNAP} --resume {RESUME} --outdir {EXPERIMENTS} --data {DATA}"
!{cmd}


Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 1,
  "network_snapshot_ticks": 1,
  "metrics": [
    "fid50k_full"
  ],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "/content/drive/MyDrive/Image_making/texture/Marble_data_convert_1",
    "use_labels": false,
    "max_size": 1755,
    "xflip": false,
    "resolution": 1024
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 3,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "synthesis_kwargs": {
      "channel_base": 32768,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "cha

## extender